In [ ]:
# Sabina Hartnett

# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. 

In [4]:
#!pip install --upgrade pip

  Using cached https://files.pythonhosted.org/packages/54/eb/4a3642e971f404d69d4f6fa3885559d67562801b99d7592487f1ecc4e017/pip-20.3.3-py2.py3-none-any.whl
  Found existing installation: pip 8.1.1
    Uninstalling pip-8.1.1:
Exception:
Traceback (most recent call last):
  File "/Library/Python/2.7/site-packages/pip-8.1.1-py2.7.egg/pip/basecommand.py", line 209, in main
    status = self.run(options, args)
  File "/Library/Python/2.7/site-packages/pip-8.1.1-py2.7.egg/pip/commands/install.py", line 317, in run
    prefix=options.prefix_path,
  File "/Library/Python/2.7/site-packages/pip-8.1.1-py2.7.egg/pip/req/req_set.py", line 726, in install
    requirement.uninstall(auto_confirm=True)
  File "/Library/Python/2.7/site-packages/pip-8.1.1-py2.7.egg/pip/req/req_install.py", line 746, in uninstall
    paths_to_remove.remove(auto_confirm)
  File "/Library/Python/2.7/site-packages/pip-8.1.1-py2.7.egg/pip/req/req_uninstall.py", line 115, in remove
    renames(path, new_path)
  File "/Library/Py

In [3]:
!pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package

  Cloning git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /private/var/folders/gp/7nv5n7813r91hvhzxdz7t6cm0000gn/T/pip-req-build-06vt62ac
  Created wheel for lucem-illud: filename=lucem_illud-8.0.1-py3-none-any.whl size=34914 sha256=57bf92f19aa1d5204de0f0bcbb13d04a3edadc603f4a744f256f6de42eab5cfa
  Stored in directory: /private/var/folders/gp/7nv5n7813r91hvhzxdz7t6cm0000gn/T/pip-ephem-wheel-cache-_cyi0bys/wheels/26/66/59/2c69dc2c9856ccf2d3eb274e5754cbb361d91e0a577f63c61d
Successfully built lucem-illud


If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells. 

In [4]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/anaconda3/lib/python3.8/site-packages/en_core_web_sm -->
/opt/anaconda3/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [29]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [30]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [31]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"X-sthQpAEJkAAcU8zrUAAABO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":991020438,"wgRevisionId":991020438,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All articles needing expert attention","Sociology ar

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [32]:
 dir(wikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains. 



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [33]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia































Content analysis

From Wikipedia, the free encyclopedia



Jump to navigation
Jump to search
This article needs attention from an expert


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [34]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag. 

In [35]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is the study of documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in content analysis to auto

Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [36]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is the study of documents and...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   Quantitative content analysis highlights frequ...
7   Siegfried Kracauer provides a critique of quan...
8   More generally, content analysis is research u...
9   By having contents of communication available ...
10  Computer-assisted analysis can help with large...
11  Robert Weber notes: "To make valid inferences ...
12  There are five types of texts in content analy...
13  Over the years, content analysis has been appl...
14  In recent times, particularly with the advent ...
15  Quantitative content analysis has enjoyed a re...
16  Content analysis can also be described as stud...
17  Manifest content is read

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators. 

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [37]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [38]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [39]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [351]:
#declare website as variable
wiki_proud_boys = 'https://en.wikipedia.org/wiki/Proud_Boys'

In [352]:
#get content from that page
PBwikiContentRequest = requests.get(wiki_proud_boys)
# then use dir to see the list of attributes and functions:
dir(PBwikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [353]:
#use beautifulsoup to clean and see whats there --> using text attribute
PBwikiContentSoup = bs4.BeautifulSoup(PBwikiContentRequest.text, 'html.parser')
#The below print has a bunch of empty new lines... see next cell for more succinct printing
#print(wikiContentSoup.text[:200])

In [354]:
#create pandas dataframe of all paragraphs on the page:
PBcontentParagraphs = []
PBcontentPTags = PBwikiContentSoup.body.findAll('p')
for pTag in PBcontentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    PBcontentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

In [355]:
#convert to a DataFrame
PBcontentParagraphsDF = pandas.DataFrame({'paragraph-text' : PBcontentParagraphs})
print(PBcontentParagraphsDF)

#now we have a 78 row DF of the paragraphs in the wikipedia page for the Proud Boys

                                       paragraph-text
0                                                  \n
1                                                  \n
2   The Proud Boys is a far-right, neo-fascist, an...
3   The group believes men and Western culture are...
4   Gavin McInnes co-founded Vice magazine in 1994...
..                                                ...
73  Members of the Proud Boys can be identified by...
74  Since the early days of the group, Proud Boys ...
75  In 2017, Fred Perry's CEO John Flynn denounced...
76  Proud Boys commonly wear red MAGA hats to rall...
77  "6MWE" is an antisemitic slogan that appears o...

[78 rows x 1 columns]


In [356]:
PB_specific_contentParagraphs = []

for i in range(len(PBcontentParagraphsDF['paragraph-text'])):
    par = PBcontentParagraphsDF['paragraph-text'][i]
    if 'proudboys' in par:
        PB_specific_contentParagraphs.append(par)
        
PB_specificcontentParagraphsDF = pandas.DataFrame({'#proudboys' : PB_specific_contentParagraphs})
        
PB_specificcontentParagraphsDF

,#proudboys
0,"In early October 2020, a campaign from the LGB..."
1,"In response to the tweets, Proud Boys leader T..."
2,During the 2020 presidential campaign in Octob...


In [357]:
type(PB_specific_contentParagraphs)
for par in PB_specific_contentParagraphs:
    print(par)

In early October 2020, a campaign from the LGBT community on Twitter, seemingly started by actor George Takei, was launched to 'reclaim' the hashtag #proudboys from hate and bigotry, to inclusion and joy. The hashtag was previously used by the group to identify bigoted content, but it begun to be taken over by positive messages celebrating LGBT families and communities, with photos of gay friends, families and memes.

In response to the tweets, Proud Boys leader Tarrio officially commented that it was hysterical and that "this isn't something that's offensive to us. It's not an insult. We aren't homophobic. We don't care who people sleep with. People think it's going to bother us. It doesn't." On largely-conservative social media app Parler, messages posted by members contained chiefly hateful comments such as "fags, can't stand gay people...should be illegal." Forbes has also reported Tarrio had posted opposing comments on Parler, describing how he believes "the left was attempting to

In [368]:
import numpy as np
#dates = r'^January$, ^February$, ^March$, ^April$, ^May$, ^June$, ^July$, ^August$, ^September$, ^October$, ^November$, ^December$'
findCapitalized = r'[A-Z]\w+'

tester = r"(\w+) (\w+)"
cap_per_paragraph = []
#iterate through the paragraphs which reference proud boys

def find_unique_words(contentPs):
    findCapitalized = r'[A-Z]\w+'

    tester = r"(\w+) (\w+)"
    cap_per_paragraph = []
    for par in contentPs:
        for word in par.split():
            capitalResults = re.search(findCapitalized, word)
            #find other possible proper nouns referenced

            if capitalResults != None and len(capitalResults[0]) > 3:
                #include positive matches and words with length > 3 (excluding 'in', 'we', 'the', etc.)

                cap_per_paragraph.append(capitalResults[0])
    unique_capital = np.unique(cap_per_paragraph)
    return unique_capital

In [369]:
unique_PB = find_unique_words(PB_specific_contentParagraphs)
print(unique_PB)

['Alaska' 'Arizona' 'Boys' 'Cloud' 'Democratic' 'During' 'Election'
 'Enrique' 'Florida' 'Forbes' 'George' 'Google' 'Iranian' 'LGBT' 'Miami'
 'October' 'Officials' 'Parler' 'Pennsylvania' 'People' 'Post' 'Proud'
 'Russia' 'Takei' 'Tarrio' 'Times' 'Trump' 'Twitter' 'Washington']


In [370]:
#now try this for all contents
findCapitalized = r'[A-Z]\w+'

tester = r"(\w+) (\w+)"
cap_per_paragraph = []

unique_Overall = find_unique_words(PBcontentParagraphs)
print(unique_Overall)

['AIRC' 'According' 'Admiral' 'African' 'Afro' 'After' 'Agent' 'Aire'
 'Alabama' 'Aladdin' 'Alan' 'Alaska' 'Alex' 'Alliance' 'Also' 'Although'
 'Amendment' 'America' 'American' 'Analysis' 'Andy' 'Angeles' 'Another'
 'Anti' 'Antifa' 'April' 'Arizona' 'Armed' 'Army' 'Asanuma' 'Ashbury'
 'Asian' 'Assessment' 'Atomwaffen' 'Attorney' 'August' 'Augusto'
 'Augustus' 'Austin' 'Autonomous' 'Avowed' 'Axtell' 'Based' 'Beast'
 'Because' 'Beirich' 'Belt' 'Berger' 'Berkeley' 'Betancur' 'Better'
 'Biden' 'Biggs' 'Bill' 'Black' 'Blair' 'Boys' 'Brent' 'British'
 'Brooklyn' 'Bruce' 'Bryan' 'Burke' 'CIAC' 'CONSEQUENCES' 'COVID'
 'California' 'Calling' 'Canada' 'Canadian' 'Capitol' 'Carlos' 'Carolina'
 'Caucasian' 'Center' 'Cernovich' 'Chapman' 'Chapter' 'Charge'
 'Charlottesville' 'Chase' 'Chris' 'Church' 'City' 'Clark' 'Cloud' 'Club'
 'Coleman' 'Colorado' 'Columbia' 'Commander' 'Communist' 'Company'
 'Concord' 'Confederate' 'Conference' 'Congresswoman' 'Cornwallis'
 'Corporal' 'County' 'Covid' 'DNAinfo'

In [ ]:
#some interesting unique capitalized words here!! 
#my interest in doing this was to draw out *proper nouns* to see if there is general affiliation
#or shared interest or ties between some surprising people and the group
#I think it would be cool to count the unique words as well!


# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [48]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


In [167]:
#print(contentPTags)

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph. 

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [50]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,Quantitative content analysis highlights frequ...,https://en.wikipedia.org/wiki/Content_analysis,6
7,Siegfried Kracauer provides a critique of quan...,https://en.wikipedia.org/wiki/Content_analysis,7
8,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,8
9,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [51]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [52]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0,None,None
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1,None,None
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2,None,None
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3,None,None
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4,None,None
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5,None,None
6,Quantitative content analysis highlights frequ...,https://en.wikipedia.org/wiki/Content_analysis,6,None,None
7,Siegfried Kracauer provides a critique of quan...,https://en.wikipedia.org/wiki/Content_analysis,7,None,None
8,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,8,None,None
9,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,9,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [168]:
#wikipedia_base_url = 'https://en.wikipedia.org'

PBotherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(PBcontentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        PBotherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(PBotherPAgeURLS[:10])
#print(PBcontentPTags)


[('https://en.wikipedia.org/wiki/Political_violence', 2, 'political violence'), ('https://en.wikipedia.org/wiki/Taki%27s_Magazine', 2, "Taki's Magazine"), ('https://en.wikipedia.org/wiki/Vice_Media', 2, 'Vice Media'), ('https://en.wikipedia.org/wiki/Gavin_McInnes', 2, 'Gavin McInnes'), ('https://en.wikipedia.org/wiki/Proud_of_Your_Boy', 2, 'Proud of Your Boy'), ('https://en.wikipedia.org/wiki/Aladdin_(2011_musical)', 2, 'Aladdin'), ('https://en.wikipedia.org/wiki/Alt-right', 2, 'alt-right'), ('https://en.wikipedia.org/wiki/Race_and_ethnicity_in_the_United_States', 2, 'race'), ('https://en.wikipedia.org/wiki/White_supremacy', 2, 'white supremacist'), ('https://en.wikipedia.org/wiki/Unite_the_Right_rally', 2, 'Unite the Right rally')]


In [177]:
#testing access to linked urls
for i in range(3):
    print(PBotherPAgeURLS[i][0])

https://en.wikipedia.org/wiki/Political_violence
https://en.wikipedia.org/wiki/Taki%27s_Magazine
https://en.wikipedia.org/wiki/Vice_Media


In [372]:
#adding in one more 'recursive' layer (visiting links of links)
#my understanding of recursive requires a base case... not positive what that would be for this
#see below for attempt at recursion! I think it works... can just ignore this box if yes

PBlinkedotherPAgeURLS = []

for i in range(4): #len(PBotherPAgeURLS)
    #print("working on", PBotherPAgeURLS[i][0])
    wikiContentRequest = requests.get(PBotherPAgeURLS[i][0])
    wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
    contentPTags = wikiContentSoup.body.findAll('p')

    for paragraphNum, pTag in enumerate(contentPTags):
        #we only want hrefs that link to wiki pages
        tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
        for aTag in tagLinks:
            #We need to extract the url from the <a> tag
            relurl = aTag.get('href')
            linkText = aTag.text
            #wikipedia_base_url is the base we can use the urllib joining function to merge them
            #Giving a nice structured tupe like this means we can use tuple expansion later

            PBlinkedotherPAgeURLS.append((
                urllib.parse.urljoin(wikipedia_base_url, relurl),
                paragraphNum,
                linkText,
            ))
            
print(PBlinkedotherPAgeURLS[:10])
#print(PBcontentPTags)

[('https://en.wikipedia.org/wiki/Violence', 0, 'violence'), ('https://en.wikipedia.org/wiki/Government', 0, 'governments'), ('https://en.wikipedia.org/wiki/State_(polity)', 0, 'state'), ('https://en.wikipedia.org/wiki/War', 0, 'war'), ('https://en.wikipedia.org/wiki/Non-state_actor', 0, 'non-state actors'), ('https://en.wikipedia.org/wiki/Police_brutality', 0, 'police brutality'), ('https://en.wikipedia.org/wiki/Genocide', 0, 'genocide'), ('https://en.wikipedia.org/wiki/Rebellion', 0, 'rebellion'), ('https://en.wikipedia.org/wiki/Treason', 0, 'treason'), ('https://en.wikipedia.org/wiki/Famine', 0, 'famine')]


In [373]:
#attempting recursion
linkedotherPAgeURLS = []

def append_web_links(link, otherURLS, recursions, max_recursions, num_links_follow):
    #if either of these base cases: end
    if link == 'NULL':
        return
    elif recursions > max_recursions:
        return 
    
    else:
        #else iterate
        tempURLS = []
        #to save time.. pick a low num_links_follow
        for i in range(num_links_follow): #len(otherURLS)
            #print("working on", PBotherPAgeURLS[i][0])
            wikiContentRequest = requests.get(otherURLS[i][0])
            wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
            contentPTags = wikiContentSoup.body.findAll('p')

            for paragraphNum, pTag in enumerate(contentPTags):
                #we only want hrefs that link to wiki pages
                tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
                for aTag in tagLinks:
                    #We need to extract the url from the <a> tag
                    relurl = aTag.get('href')
                    linkText = aTag.text
                    #wikipedia_base_url is the base we can use the urllib joining function to merge them
                    #Giving a nice structured tupe like this means we can use tuple expansion later

                    tempURLS.append((
                        urllib.parse.urljoin(wikipedia_base_url, relurl),
                        paragraphNum,
                        linkText,
                    ))
            linkedotherPAgeURLS.append(tempURLS)
            return append_web_links(otherURLS[i][0], tempURLS, recursions + 1, max_recursions, num_links_follow)

#print(PBlinkedotherPAgeURLS[:10])
#print(PBcontentPTags)

In [385]:
append_web_links(wiki_proud_boys, PBotherPAgeURLS, 0, 5, 2)
print(linkedotherPAgeURLS[1][:2])

[('https://en.wikipedia.org/wiki/World_Health_Organization', 1, 'World Health Organization'), ('https://en.wikipedia.org/wiki/Power_(social_and_political)', 1, 'power')]


In [386]:
#here we add the paragraph (contents) and paragraph number to the contentDF
PBcontentParagraphsDF['source'] = [wiki_proud_boys] * len(PBcontentParagraphsDF['paragraph-text'])
PBcontentParagraphsDF['paragraph-number'] = range(len(PBcontentParagraphsDF['paragraph-text']))
PBcontentParagraphsDF

,paragraph-text,source,paragraph-number
0,\n,https://en.wikipedia.org/wiki/Proud_Boys,0
1,\n,https://en.wikipedia.org/wiki/Proud_Boys,1
2,"The Proud Boys is a far-right, neo-fascist, an...",https://en.wikipedia.org/wiki/Proud_Boys,2
3,The group believes men and Western culture are...,https://en.wikipedia.org/wiki/Proud_Boys,3
4,Gavin McInnes co-founded Vice magazine in 1994...,https://en.wikipedia.org/wiki/Proud_Boys,4
...,...,...,...
73,Members of the Proud Boys can be identified by...,https://en.wikipedia.org/wiki/Proud_Boys,73
74,"Since the early days of the group, Proud Boys ...",https://en.wikipedia.org/wiki/Proud_Boys,74
75,"In 2017, Fred Perry's CEO John Flynn denounced...",https://en.wikipedia.org/wiki/Proud_Boys,75
76,Proud Boys commonly wear red MAGA hats to rall...,https://en.wikipedia.org/wiki/Proud_Boys,76


In [387]:
#Then we can add two more columns to our Dataframe and define a function to 
#parse each linked page and add its text to our DataFrame.

PBcontentParagraphsDF['source-paragraph-number'] = [None] * len(PBcontentParagraphsDF['paragraph-text'])
PBcontentParagraphsDF['source-paragraph-text'] = [None] * len(PBcontentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    PBparsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        PBparsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        PBparsDict['paragraph-number'].append(parNum)
        PBparsDict['source'].append(targetURL)
        PBparsDict['source-paragraph-number'].append(sourceParNum)
        PBparsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(PBparsDict)

for urlTuple in PBotherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    PBcontentParagraphsDF = PBcontentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)

PBcontentParagraphsDF

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,\n,https://en.wikipedia.org/wiki/Proud_Boys,0,None,None
1,\n,https://en.wikipedia.org/wiki/Proud_Boys,1,None,None
2,"The Proud Boys is a far-right, neo-fascist, an...",https://en.wikipedia.org/wiki/Proud_Boys,2,None,None
3,The group believes men and Western culture are...,https://en.wikipedia.org/wiki/Proud_Boys,3,None,None
4,Gavin McInnes co-founded Vice magazine in 1994...,https://en.wikipedia.org/wiki/Proud_Boys,4,None,None
...,...,...,...,...,...
223,The accusation that the NUJ has not been trans...,https://en.wikipedia.org/wiki/Vice_Media,76,2,Vice Media
224,"In July 2014, Vice Media announced it would be...",https://en.wikipedia.org/wiki/Vice_Media,77,2,Vice Media
225,"Following this announcement, the two music ven...",https://en.wikipedia.org/wiki/Vice_Media,78,2,Vice Media
226,After a series of articles covering the venues...,https://en.wikipedia.org/wiki/Vice_Media,79,2,Vice Media


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [193]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3707,"posts-type":false,"posts":[{"id":"639732477227892736","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/639732477227892736","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/639732477227892736\/cat-demands-hairspray-sniffs","type":"photo","date-gmt":"2021-01-08 09:00:32 GMT","date":"Fri, 08 Jan 2021 10:00:32","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1610096432,"format":"html","reblog-key":"ZbrAwRC4","slug":"cat-demands-hairspray-sniffs","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"639732477227892736\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_639732477227892736\"><iframe id=\"like_iframe_639732477227892736\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [196]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
1


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [195]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [198]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
#tumblrImageScrape('lolcats-lol-cat', 50)

Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [199]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [200]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory 

In [223]:
with open('./data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

FileNotFoundError: [Errno 2] No such file or directory: './data/Shakespeare/midsummer_nights_dream.txt'

By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory. 

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [206]:
targetDir = './data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

FileNotFoundError: [Errno 2] No such file or directory: './data/Shakespeare'

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [211]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�j9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45��%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?!IYi���U�"��y;���#��e3)�+B�&����E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ


It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [212]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [213]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [214]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [207]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [208]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [209]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [210]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [344]:
#hmmmm this seems unnecessarily long... looking forward to learning how to make this more succinct

import pandas as pd
from bs4 import BeautifulSoup

#had trouble accessing folders to shakespeare texts... so going to use the PDF
#above code for PDF needs to be run in order to start here!
PDF_read = readPDF(infoExtractionBytes)

#print(PDF_read[550:2000])
#soup = BeautifulSoup(PDF_read, 'pdf.parser')
loc_TA_info = PDF_read.find("TAs")
TA_info1 = PDF_read[loc_TA_info + 5:loc_TA_info+92]
TA_info2 = PDF_read[loc_TA_info+92:loc_TA_info+175]

list_TA_info = []
list_TA_info.append([TA_info1])
list_TA_info.append([TA_info2])

for i in range(2):
    semi = r';'
    newLine = r'\n'
    officeHours = r'Office Hours'
    email = '@uchicago.edu'
    resultS = re.search(semi, list_TA_info[i][0])
    resultNL = re.search(newLine, list_TA_info[i][0])
    resultOH = re.search(officeHours, list_TA_info[i][0])
    resultEMail = re.search(email, list_TA_info[i][0])
    
    if resultS != None:
        list_TA_info[i][0] = list_TA_info[i][0].replace(';',',')
    if resultNL != None:
        list_TA_info[i][0] = list_TA_info[i][0].replace('\n','')    
    if resultOH != None:
        list_TA_info[i][0] = list_TA_info[i][0].replace('Office Hours:','')
    if email != None:
        list_TA_info[i][0] = list_TA_info[i][0].replace('.edu','.edu,')
    
    list_TA_info[i] = list_TA_info[i][0].split(',')

col_names = ['Name', 'Email', 'Office Hours']
pd.DataFrame(list_TA_info[0:], columns = col_names)

,Name,Email,Office Hours
0,Reid McIlroy,reidmcy@uchicago.edu,Monday 11:00-1:00pm
1,Linzhuo Li,linzhuoli@uchicago.edu,Wednesday 11:00-1:00pm


### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

